In [14]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn import preprocessing
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
df = pd.read_parquet('data/market_final.parquet')

In [3]:
df_purchase = df[df['event_type'] == 'purchase']
df_purchase.head()

,event_time,event_type,product_id,category_code,sub_category,product,brand,price,user_id,user_session
83,2019-11-01 00:01:04,purchase,1005161,electronics,smartphone,smartphone,xiaomi,211.92,513351129,e6b7ce9b-1938-4e20-976c-8b4163aea11d
389,2019-11-01 00:04:51,purchase,1004856,electronics,smartphone,smartphone,samsung,128.42,562958505,0f039697-fedc-40fa-8830-39c1a024351d
528,2019-11-01 00:06:33,purchase,1801881,electronics,video,tv,samsung,488.80,557746614,4d76d6d3-fff5-4880-8327-e9e57b618e0e
530,2019-11-01 00:06:34,purchase,5800823,electronics,audio,subwoofer,nakamichi,123.56,514166940,8ef5214a-86ad-4d0b-8df3-4280dd411b47
612,2019-11-01 00:07:38,purchase,30000218,construction,tools,welding,magnetta,254.78,515240495,0253151d-5c84-4809-ba02-38ac405494e1


In [4]:
def mean(s):
    return np.mean(s)

df_monetary = df_purchase.groupby('user_id').agg({'price':'sum'})
df_monetary = df_monetary.rename(columns={'price':'monetary'}).reset_index()
df_monetary.head()

,user_id,monetary
0,225644257,40.91
1,253299396,246.85
2,256164170,113.23
3,296197073,51.46
4,299358698,154.19


In [5]:
temp_recency_df = df_purchase[['user_id','event_time']].drop_duplicates()
df_recency = temp_recency_df.groupby('user_id')['event_time'].max().reset_index()
df_recency = df_recency.rename(columns={'event_time':'recency'})
df_recency.head()

,user_id,recency
0,225644257,2019-11-12 04:19:31
1,253299396,2019-11-06 10:53:31
2,256164170,2019-11-20 08:21:14
3,296197073,2019-11-30 08:50:49
4,299358698,2019-11-09 03:21:25


In [6]:
temp_frequency_df = df_purchase[['user_id','event_time']].drop_duplicates()
df_frequency = temp_frequency_df.groupby('user_id')['event_time'].count().reset_index()
df_frequency = df_frequency.rename(columns={'event_time':'frequency'})
df_frequency.head()

,user_id,frequency
0,225644257,1
1,253299396,1
2,256164170,1
3,296197073,1
4,299358698,1


In [7]:
df_rfm = pd.merge(df_recency, df_frequency, how='left', on='user_id')
df_rfm = pd.merge(df_rfm, df_monetary, how='left', on='user_id')

In [8]:
df_rfm

,user_id,recency,frequency,monetary
0,225644257,2019-11-12 04:19:31,1,40.91
1,253299396,2019-11-06 10:53:31,1,246.85
2,256164170,2019-11-20 08:21:14,1,113.23
3,296197073,2019-11-30 08:50:49,1,51.46
4,299358698,2019-11-09 03:21:25,1,154.19
...,...,...,...,...
330389,579962677,2019-11-30 23:36:38,1,1634.38
330390,579963523,2019-11-30 23:41:03,1,457.91
330391,579964521,2019-11-30 23:43:40,1,334.60
330392,579966375,2019-11-30 23:49:23,1,124.11


In [9]:
current_day = pd.to_datetime('20191101')
time_diff = df_rfm['recency'] - current_day
time_in_seconds = [x.total_seconds() for x in time_diff]
df_rfm['recency'] = time_in_seconds

In [10]:
df_rfm

,user_id,recency,frequency,monetary
0,225644257,965971.0,1,40.91
1,253299396,471211.0,1,246.85
2,256164170,1671674.0,1,113.23
3,296197073,2537449.0,1,51.46
4,299358698,703285.0,1,154.19
...,...,...,...,...
330389,579962677,2590598.0,1,1634.38
330390,579963523,2590863.0,1,457.91
330391,579964521,2591020.0,1,334.60
330392,579966375,2591363.0,1,124.11


In [11]:
def get_score(level, data):
    '''level 안의 원소들을 기준으로 기준을 나누어 data의 점수를 부여
    리스트 형태로 반환'''
    score = []
    
    for j in range(len(data)):
        for i in range(len(level)):
            if data[j] <= level[i]:
                score.append(i+1)
                break
            elif data[j] > max(level):
                score.append(len(level)+1)
                break
            else:
                continue
    return score

def get_rfm_grade(df, num_class, rfm_tick_point, rfm_col_map, suffix=None):
    
    # 예외처리
    if not isinstance(df, pd.DataFrame):
        print('df must be pandas.DataFrame')
        return
    
    if isinstance(rfm_tick_point, dict) == False or isinstance(rfm_col_map, dict) == False:
        print('rfm_tick_point & rfm_col_map must be dict')
        return
    
    if len(rfm_col_map) != 3:
        print(f'rfm_col_map must have three keys. \n now : {len(rfm_col_map)}')
        return
    
    if len(rfm_tick_point) != 3:
        print(f'rfm_tick_point must have three keys. \n now : {len(rfm_tick_point)}')
        return
        
    if set(rfm_tick_point.keys()) != set(rfm_col_map.keys()):
        print('rfm_tick_point and rfm_col_map must have the same key')
        return
    
    if not set(rfm_col_map.values()).issubset(set(df.columns)):
        print(f'{set(rfm_col_map.values())-set(df.columns)} must be in the columns')
        return
    
    for k, v in rfm_tick_point.items():
        if isinstance(v, str):
            if not v in ['quantile', 'min_max']:
                print(f'{k} must be either "quantile" or "min_max"')
                return
        elif isinstance(v, list) or isinstance(v, tuple):
            if len(v) != num_class-1:
                print(f'There must be {num_class-1} elements in the list (tuples) corresponding to {k}.')
                return
            
    if suffix:
        if not isinstance(suffix, str):
            print('the suffix must be "str"')
            return
    
    # 점수 부여    
    for k, v in rfm_tick_point.items():
        if isinstance(v, str):
            if v == 'quantile':
                scale = preprocessing.StandardScaler() # 데이터 표준화 함수 지정
                temp_data = np.array(df[rfm_col_map[k]]) # numpy배열로 변환
                temp_data = temp_data.reshape((-1,1)) # 스케일링을 위해 1차원 배열 -> 2차원으로 변환
                temp_data = scale.fit_transform(temp_data) # 데이터 평균 0, 표준편차 1 갖도록 스케일링
                temp_data = temp_data.squeeze() # 데이터를 다시 1차원으로 변환
                
                quantiles_level = np.linspace(0,1,num_class+1)[1:-1] # 0과 1 사이 분위수를 num_class만큼 생성
                quantiles = [] # 분위수를 담을 리스트
                for x in quantiles_level:
                    quantiles.append(np.quantile(temp_data, x)) # 분위수 리스트에 삽입
            else: # 최대 최소값 경우
                temp_data = np.array(df[rfm_col_map[k]]) # numpy배열로 변환
                quantiles = np.linspace(np.min(temp_data), np.max(temp_data), num_class+1)[1:-1] # 최대 최소값 사이 등간격으로 분할
        
        # 직접 구분값을 넣어주는 경우
        else:
            temp_data = np.array([df[rfm_col_map[k]]]) # numpy배열로 변환
            quantiles = v # 직접 구분값 넣어주기
        
        score = get_score(quantiles, temp_data) # 앞서 정의한 quantiles를 기준으로 한 점수리스트 반환
        new_col_name = rfm_col_map[k]+'_'+k # 점수값을 담는 변수의 이름
        if suffix:
            new_col_name = rfm_col_map[k]+'_'+suffix
        df[new_col_name] = score
    return df
            
            

In [29]:
rfm_tick_point={'R':'min_max','F':'min_max','M':'min_max'}
rfm_col_map={'R':'recency','F':'frequency','M':'monetary'}

df_result = get_rfm_grade(df=df_rfm, num_class=5, rfm_tick_point=rfm_tick_point, rfm_col_map=rfm_col_map)

In [30]:
df_result[['user_id','recency_R','frequency_F','monetary_M']]

,user_id,recency_R,frequency_F,monetary_M
0,225644257,2,1,1
1,253299396,1,1,1
2,256164170,4,1,1
3,296197073,5,1,1
4,299358698,2,1,1
...,...,...,...,...
330389,579962677,5,1,1
330390,579963523,5,1,1
330391,579964521,5,1,1
330392,579966375,5,1,1


In [31]:
print(df_result['recency_R'].value_counts(), 
      df_result['frequency_F'].value_counts(),
      df_result['monetary_M'].value_counts(), sep='\n\n')

3    126253
5     68818
4     49433
2     44808
1     41082
Name: recency_R, dtype: int64

1    330367
2        24
3         2
5         1
Name: frequency_F, dtype: int64

1    330310
2        69
3        12
4         2
5         1
Name: monetary_M, dtype: int64


In [43]:
# f, axes = plt.subplots(2, 2, figsize=(8, 6), sharex=True)
# sns.distplot(df_result['recency_R'], color='blue', label='recency_R', ax=axes[0, 0])
# sns.distplot(df_result['frequency_F'], color='red', label='frequency_F', ax=axes[0, 1])
# sns.distplot(df_result['monetary_M'], color='green', label='monetary_M', ax=axes[1, 0])
# plt.legend(title="RFM")
# plt.show()